# Sequential removal of links and resiliency testing

### Overview
The notebook iteratively removes the links (edges) in the FTOT road network in order of importance to create distinct disruption scenarios and re-runs FTOT to determine the new optimal solutions and costs.

Importance is measured by:

- the sum of __betweenness centrality__ of a link's beginning and ending points __OR__   
- the __volume__ of background vehicle flows on a link.

The notebook outputs (i) CSV files with information on the links removed and scenario results and (ii) interactive HTML report generated with RMarkdown. Note that scenario costs in the outputs are the minimized FTOT objective value, which is based on impeded transport cost, facility build costs, and unmet demand penalties.

### Instructions
_Important: Before running this notebook, follow instructions the repository's README to (i) set up and activate the Python environment and (ii) run a baseline FTOT scenario._

__(1) Update parameters__ in the cell labeled Step 1:  
- Baseline scenario name and path  
- Measure of importance (volume or betweeness centrality)  
- Number of disruption steps  
- True/False toggle to export maps for each disruption scenario  
     
__(2) Run all cells__ by going to the top menu bar > Cell > Run All.

__(3) Review outputs__:
- [...]
- [...]

_The notebook may take several hours to run depending on the scenario size and number of disruption steps._

### Assumptions

- You are working in a Python 3.x environment for this notebook. Refer to the README in this repository for setup instructions.
- You have access to a ArcGIS license server if necessary
- A baseline FTOT scenario was run with Network Density Reduction (NDR_On set to False in the scenario XML file) and with the swapped ftot_routing.py file from this repository.

## Step 0: Load Dependencies

In [1]:
import pandas as pd
import geopandas as gpd
import sqlalchemy
import networkx as nx
import os
import pickle
import momepy # for conversion from geopandas GeoDataFrame to networkX Graph
import subprocess
import shutil
import webbrowser
import resiliency_disruptions
from osgeo import ogr

PYTHON = r"C:\FTOT\python3_env\python.exe"
FTOT = r"C:\FTOT\program\ftot.py"

## Step 1: Set User-Defined Parameters (USER INPUT REQUIRED)

In [2]:
# Uses Reference Scenario 7 as an example.
# Modify `scen_name` and `scen_path` for your scenario.
scen_name = 'rs7_capacity'
scen_path = r'C:\FTOT\scenarios\reference_scenarios\rs7_capacity'

# Enter disrupt_type 'BC' for betweeness centrality or 'V' for volume 
# Note: if background flows were not enabled in the baseline scenario,
# the notebook will automatically switch to BC.
disrupt_type = 'V'

# Enter the number of disruption scenarios to generated
# Recommend at least 25
disrupt_steps = 5 

# Set the variable `MAKE_MAPS` to `True` to output maps for each disruption scenario
# Note this will increase runtime
MAKE_MAPS = False

## Step 2: Calculate Importance Metrics

In [3]:
if disrupt_type == 'BC':
    
    # Read in prepared betweeness centrality and road network graph data
    # If these don't exist, the following steps will create them
    picklename = os.path.join(scen_path, 'BetweenessG.pickle')
    if os.path.exists(picklename):
        file = open(picklename, 'rb')
        betweenness_dict_road = pickle.load(file)
        G_road = pickle.load(file)
    
    # Run betweenness centrality on the NetworkX graph
    # Note: This step might take several minutes to a few hours   
    elif not os.path.exists(picklename):
        G_road = resiliency_disruptions.read_gdb(os.path.join(scen_path, 'main.gdb'), 'road')
        print('Running Betweenness Centrality calculations. This might take more than 20 minutes.')
        betweenness_dict_road = nx.betweenness_centrality(G_road, normalized=False, weight='Length')
        print('Completed Betweenness Centrality calculations.')
        
        # Save with pickle
        # Upon load, this pickle will contain the network G_road and the betweenness centrality dict and 
        with open(picklename, 'wb') as handle:
            pickle.dump(betweenness_dict_road, handle)
            pickle.dump(G_road, handle)

## Step 3: Associate Importance Metrics with Edges

In [4]:
# Read in FTOT data
print('Reading in {}'.format(scen_path))

db_name = 'main.db'
db_path = 'sqlite:///' + os.path.join(scen_path, db_name)
engine = sqlalchemy.create_engine(db_path)

table_name = 'networkx_edges'
nx_edges = pd.read_sql_table(table_name, engine)

table_name = 'networkx_nodes'
nx_nodes = pd.read_sql_table(table_name, engine)

table_name = 'optimal_variables'
optimal_vars = pd.read_sql_table(table_name, engine)

Reading in C:\FTOT\scenarios\reference_scenarios\rs7_capacity


In [5]:
# Check whether scenario has background flow data
# volume column in DB is filled with NULL if no
# Automatically revert to betweeness centrality if no background flow data
# TODO: Confirm want to switch if ANY are null?
BACKGROUND_FLOWS = pd.isna(nx_edges['volume']).any()
if BACKGROUND_FLOWS:
    print('Background flows confirmed')
elif disrupt_type == 'V' and not BACKGROUND_FLOWS:
    print('WARNING: Network does not have background flows.')
    print('Switching importance measure to betweenness centrality.')
    disrupt_type = 'BC'
else:
    print('Scenario does not have background flows. Proceeding with disrupt type BC.')

Background flows confirmed


In [9]:
# Load road network into Graph
#G_road_orig_label = resiliency_disruptions.read_gdb(os.path.join(scen_path, 'main.gdb'), 'road')

# Get shape_x and shape_y
#road_orig_label_nodes = list(G_road_orig_label.nodes)
#node_shape_df_road = pd.DataFrame(road_orig_label_nodes)

In [6]:
if disrupt_type == 'BC':
    
    # Get shape_x and shape_y
    road_orig_label_nodes = list(G_road.nodes)
    node_shape_df_road = pd.DataFrame(road_orig_label_nodes)
    
    # Make the betweenness_centrality values as the framework to join in shape_x, shape_y, and node_id
    bc_df_road = pd.DataFrame.from_dict(betweenness_dict_road, orient = 'index')
    bc_df_road = bc_df_road.rename(columns = {0: 'BC'})
    
    bc_shape_df_road = pd.concat([bc_df_road, node_shape_df_road], axis = 1)
    bc_shape_df_road = bc_shape_df_road.rename(columns = {0: 'shape_x', 1: 'shape_y'})
    
    # Now add node_id from networkx_nodes, using pandas merge with left join
    # Use both shape_x and shape_y to identify the nodes correctly

    bc_node_df = pd.merge(bc_shape_df_road, nx_nodes, on = ['shape_x', 'shape_y'], how = 'left')
    
    # Now use this dataframe to populate a dataframe of edges
    # We will want the following from networkx_edges:
    # edge_id, from_node_id, to_node_id, mode_source, miles, mode_source_oid
    # Then using the node_id column in the new bc_node_df, add these:
    # from_node_BC, to_node_BC
    # and sum those for sum_node_BC
    # TODO: can likely skip this step if using 'V' not 'BC'
    merge_from = pd.merge(nx_edges, bc_node_df[['BC', 'node_id']],
                          left_on = 'from_node_id',
                          right_on = 'node_id',
                          how = 'left')
    merge_from = merge_from.rename(columns = {'BC': 'from_node_BC'})

    merge_to = pd.merge(merge_from, bc_node_df[['BC', 'node_id']],
                        left_on = 'to_node_id',
                        right_on = 'node_id',
                        how = 'left')
    merge_to = merge_to.rename(columns = {'BC': 'to_node_BC'})

    # Sum the BC values

    merge_to['sum_BC'] = merge_to.filter(like = "node_BC").sum(axis = 1)

In [7]:
if disrupt_type == 'V':
    merge_to = nx_edges.copy()

In [8]:
# Select optimal_vars DB columns to keep 
use_opt_vars = ['variable_type',
               'var_id',
               'variable_value',
                'variable_name',
                'nx_edge_id',
                'mode_oid',
                'converted_capacity',
                'converted_volume'
               ]

merge_opt = pd.merge(merge_to, optimal_vars[use_opt_vars],
                     left_on = 'edge_id',
                     right_on = 'nx_edge_id',
                     how = 'left')

merge_opt.head()

,edge_id,from_node_id,to_node_id,artificial,mode_source,mode_source_oid,length,route_cost_scaling,capacity,volume,...,urban,limited_access,variable_type,var_id,variable_value,variable_name,nx_edge_id,mode_oid,converted_capacity,converted_volume
0,1,332,1887,2,road,282,3.541860,1.0,NaN,NaN,...,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,399,1888,2,road,349,0.275609,1.0,NaN,NaN,...,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,438,1889,2,road,388,0.087752,1.0,NaN,NaN,...,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,448,1890,2,road,398,0.129394,1.0,NaN,NaN,...,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,452,1891,2,road,402,0.238547,1.0,NaN,NaN,...,-9999.0,-9999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Create ranked lists of edges to remove
# (1) Keep only edges in the optimal solution
# (2) Sort by sum_BC or volume
# (3) Keep the columns we need
# (4) Reset the index to assign rank

# Note: in resiliency_disruptions.disrupt_network, the edges_remove DataFrame is sorted again by 'V' or 'BC'

use_cols = ['edge_id', 'from_node_id', 'to_node_id', 'length', 'capacity', 'volume', 'sum_BC',
            'variable_type', 'variable_value', 'variable_name', 'nx_edge_id', 'mode_oid', 'converted_capacity',
            'converted_volume']

if disrupt_type == 'V':
    edges_remove = merge_opt[merge_opt['variable_value'] > 0].sort_values(by = 'volume', ascending = False).filter(items = use_cols).reset_index()
elif disrupt_type == 'BC':
    edges_remove = merge_opt[merge_opt['variable_value'] > 0].sort_values(by = 'sum_BC', ascending = False).filter(items = use_cols).reset_index()

edges_remove.to_csv(os.path.join(scen_path, 'Edges_to_Remove.csv'), index = False)

edges_remove.head()

,index,edge_id,from_node_id,to_node_id,length,capacity,volume,variable_type,variable_value,variable_name,nx_edge_id,mode_oid,converted_capacity,converted_volume
0,5482,5483,4141,7187,0.071771,69446.587921,47941.5,Edge,22679.618,Edge_10974,5483.0,249096.0,1.666718e+06,1150596.0
1,3029,3030,3094,3643,0.318439,45000.000000,41687.0,Edge,75443.128,Edge_6067,3030.0,80088.0,1.080000e+06,1000488.0
2,12538,12539,7167,1898,0.031103,47745.379525,40784.0,Edge,43134.521,Edge_25085,12539.0,249072.0,1.145889e+06,978816.0
3,60,61,1898,7194,0.828394,47745.379525,40784.0,Edge,43134.521,Edge_129,61.0,249518.0,1.145889e+06,978816.0
4,14861,14862,8218,8215,1.454485,47739.607172,38842.5,Edge,43134.521,Edge_29731,14862.0,270845.0,1.145751e+06,932220.0


## Step 4: Create Scenarios, Disrupt Edges, and Run FTOT

Create disrupted network by copying everything in `scen_path` to a new directory. Then overwrite the `networkx_edges` tables in that main.db with the disrupted versions.

##### Assumptions:

  1. ArcGIS Pro is installed.
  2. The FTOT version being used has been modified according to the `README` in this directory.


In [10]:
# Make new scenarios
resiliency_disruptions.make_disruption_scenarios(disrupt_type, disrupt_steps, scen_path)

# Apply disruption
resiliency_disruptions.disrupt_network(disrupt_type, disrupt_steps, scen_path, edges_remove)

Prepared 5 scenarios based on rs7_capacity
Disrupted 5 scenarios


In [11]:
# Begin running O through M steps of FTOT on the disupted scenarios
# This may take several hours, depending on size of the network and number of disruption scenarios
results = resiliency_disruptions.run_o_steps(disrupt_type, disrupt_steps, scen_path, PYTHON, FTOT, MAKE_MAPS)

Running o1 for disrupt01
Running o2 for disrupt01
Running p for disrupt01
Running d for disrupt01
Preparing to search over o2_log_2024_04_12_15-03-05.log
  disrupt_step unmet_demand unmet_cost nedge total_cost
0           01            0          0   497      2,412
Running o1 for disrupt02
Running o2 for disrupt02
Running p for disrupt02
Running d for disrupt02
Preparing to search over o2_log_2024_04_12_15-06-17.log
  disrupt_step unmet_demand unmet_cost nedge total_cost
0           02            0          0   493      2,414
Running o1 for disrupt03
Running o2 for disrupt03
Running p for disrupt03
Running d for disrupt03
Preparing to search over o2_log_2024_04_12_15-09-29.log
  disrupt_step unmet_demand unmet_cost nedge total_cost
0           03            0          0   488      2,414
Running o1 for disrupt04
Running o2 for disrupt04
Running p for disrupt04
Running d for disrupt04
Preparing to search over o2_log_2024_04_12_15-12-51.log
  disrupt_step unmet_demand unmet_cost nedge tot

## Step 5: Generate Disruption Result Report

In [ ]:
# TODO: bug in the handoff here
R_Process = subprocess.Popen(['Rscript.exe', 'compile_report.R', scen_path, str(BACKGROUND_FLOWS), disrupt_type],
                 stdout = subprocess.PIPE, stderr = subprocess.PIPE)

here = os.getcwd()

# move rendered HTML file when complete to scen_path, will replace any existing file
if not os.path.exists(os.path.join(here, 'Disruption_Results.html')):
    print("OUTPUT FILE ERROR: Disruption_Results.html could not be found")
    raise Exception("OUTPUT FILE ERROR: Disruption_Results.html could not be found")
shutil.move(os.path.join(here, 'Disruption_Results.html'), os.path.join(scen_path, 'Disruption_Results_' +
                                                                    http://localhost:8888/notebooks/github/FTOT-Resilience-Link_Removal/link_removal/Conduct_Link_Removal.ipynb#    disrupt_type + '_' + str(disrupt_steps) +
                                                                        '.html'))

In [15]:
webbrowser.open('file://' + os.path.realpath(os.path.join(scen_path, 'Disruption_Results_' + disrupt_type +
                                                          '_' + str(disrupt_steps) + '.html')))

True